In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 22918085
paper_name = 'ozaydin_keasling_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/1-s2.0-S109671761200081X-mmc1.xlsx', sheet_name='Color scores of all deletions')

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5153 x 9


In [7]:
original_data.head()

,record no.,ORF name,Strain,Batch,Plate,Row,Col,Comment,Color
0,338,YAL068C,BY4741,01_1,1,A,2,NaN,0
1,339,YAL067C,BY4741,01_1,1,A,3,NaN,0
2,340,YAL066W,BY4741,01_1,1,A,4,NaN,0
3,341,YAL065C,BY4741,01_1,1,A,5,NaN,0
4,345,YAL062W,BY4741,01_1,1,A,6,NaN,0


In [8]:
original_data['orf'] = original_data['ORF name'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [12]:
original_data.loc[original_data['orf']=='YLR287-A','orf'] = 'YLR287C-A'

In [13]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [record no., ORF name, Strain, Batch, Plate, Row, Col, Comment, Color, orf]
Index: []


In [15]:
original_data['data'] = pd.to_numeric(original_data['Color'], errors='coerce')

In [16]:
original_data.set_index('orf', inplace=True)

In [17]:
original_data = original_data[['data']].copy()

In [18]:
original_data = original_data.groupby(original_data.index).mean()

In [19]:
original_data.shape

(4944, 1)

# Prepare the final dataset

In [20]:
data = original_data.copy()

In [21]:
dataset_ids = [22230]
datasets = datasets.reindex(index=dataset_ids)

In [22]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [23]:
data.head()

dataset_id,22230
data_type,value
orf,
YAL002W,-1.0
YAL004W,-4.0
YAL005C,-3.0
YAL007C,3.0
YAL008W,2.0


## Subset to the genes currently in SGD

In [24]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 24


In [25]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,22230
,data_type,value
gene_id,orf,
2,YAL002W,-1.0
1863,YAL004W,-4.0
4,YAL005C,-3.0
5,YAL007C,3.0
6,YAL008W,2.0


# Normalize

In [26]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [27]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [28]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

dataset_id      22230          
data_type       value    valuez
gene_id orf                    
2       YAL002W  -1.0 -0.717958
1863    YAL004W  -4.0 -2.768279
4       YAL005C  -3.0 -2.084839
5       YAL007C   3.0  2.015804
6       YAL008W   2.0  1.332364

# Print out

In [29]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [32]:
# from IO.save_data_to_db3 import *

In [34]:
# save_data_to_db(data_all, paper_pmid)